In [ ]:
# Load the CIFAR-10 dataset
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((384, 384)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Define the model
model = torchvision.models.VisionTransformer(
    image_size=32, patch_size=8, num_layers=6, num_heads=8, hidden_dim=512, mlp_dim=10
)

model = torchvision.models.vit_b_16(weights=torchvision.models.ViT_B_16_Weights.IMAGENET1K_SWAG_E2E_V1)


# Replace the model's final linear layer
model.classifier = nn.Linear(in_features=2048, out_features=10)


# # Load the pretrained weights
# model.load_state_dict(torch.load('pretrained_weights.pth'))

# Freeze the parameters of the pretrained model
# for param in model.parameters():
#     param.requires_grad = False
    
# # Unfreeze the final layer and set it to train mode
# for param in model.classifier.parameters():
#     param.requires_grad = True

model.train()

# for idx, tensor in enumerate(model_mob.parameters()):
#     if idx < 173:
#         tensor.requires_grad = False

for tensor in model_mob.parameters():
    tensor.requires_grad = False

for tensor in model_mob.classifier.parameters():
    tensor.requires_grad = True

# Define the loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()

# Initialize the DataModule and Model
data_module = CIFAR10DataModule(train_dataset, val_dataset)
lightning_model = CIFAR10Model(model_mob, loss_fn, optimizer)


from pytorch_lightning.callbacks import RichProgressBar

# Define the trainer
# trainer = pl.Trainer(max_epochs=10, default_root_dir='../logs', logger=wandb_logger, callbacks=RichProgressBar(leave=True))
trainer = pl.Trainer(max_epochs=10, default_root_dir='../logs', logger=wandb_logger)


# Start training
trainer.fit(lightning_model, data_module)


# Define the PyTorch Lightning DataModule
# class CIFAR10DataModule(pl.LightningDataModule):
#     def __init__(self, train_dataset, val_dataset):
#         super().__init__()
#         self.train_dataset = train_dataset
#         self.val_dataset = val_dataset

#     # def setup(self, stage=None):
#     #     self.train_dataloader = torch.utils.data.DataLoader(self.train_dataset, batch_size=64, shuffle=True)
#     #     self.val_dataloader = torch.utils.data.DataLoader(self.val_dataset, batch_size=64, shuffle=False)

#     def train_dataloader(self):
#         return torch.utils.data.DataLoader(self.train_dataset, batch_size=128, shuffle=True, num_workers=4)

#     def val_dataloader(self):
#         return torch.utils.data.DataLoader(self.val_dataset, batch_size=128, shuffle=False, num_workers=4)